In [1]:
# ==============================================
# Task 1.3 ECG Classification - Data Augmentation
# ==============================================

# ✅ Cell 1: Imports and Setup
import numpy as np
import pandas as pd
import struct
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle

BATCH_SIZE = 64
EPOCHS = 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", DEVICE)

Using device: cuda


In [2]:
# ✅ Cell 2: Load Data Splits and Raw Data
def load_bin_file(filepath):
    signals = []
    with open(filepath, 'rb') as f:
        while True:
            length_bytes = f.read(4)
            if not length_bytes:
                break
            length = struct.unpack('i', length_bytes)[0]
            signal_bytes = f.read(length * 2)
            signal = np.frombuffer(signal_bytes, dtype=np.int16)
            signals.append(signal)
    return signals

X_train = load_bin_file('../1.1_dataset/data/X_train.bin')
X_test = load_bin_file('../1.1_dataset/data/X_test.bin')

train_indices = np.load('../1.1_dataset/data/train_indices.npy')
val_indices = np.load('../1.1_dataset/data/val_indices.npy')
y_train_split = np.load('../1.1_dataset/data/y_train_split.npy')
y_val_split = np.load('../1.1_dataset/data/y_val_split.npy')

X_train_split = [X_train[i] for i in train_indices]
X_val_split = [X_train[i] for i in val_indices]

print(f"Train: {len(X_train_split)}, Val: {len(X_val_split)}, Test: {len(X_test)}")

Train: 4943, Val: 1236, Test: 2649


In [3]:
# ✅ Cell 3: Simple Data Augmentation Functions
def amplitude_scaling(signal, scale_range=(0.9, 1.1)):
    return signal * np.random.uniform(*scale_range)

def add_noise(signal, noise_std=5.0):
    noise = np.random.normal(0, noise_std, size=signal.shape)
    return signal + noise

def time_shift(signal, max_shift=200):
    shift = np.random.randint(-max_shift, max_shift)
    return np.roll(signal, shift)


In [4]:
from scipy.signal import resample

def time_stretch(signal, stretch_range=(0.8, 1.2)):
    """Time stretch signal by resampling."""
    factor = np.random.uniform(*stretch_range)
    new_length = int(len(signal) * factor)
    stretched = resample(signal, new_length)
    # Pad or crop to original length
    if new_length > len(signal):
        return stretched[:len(signal)]
    else:
        return np.pad(stretched, (0, len(signal) - new_length), mode='constant')

def frequency_dropout(signal, drop_prob=0.1):
    """Randomly zero out some frequencies in the Fourier domain."""
    fft = np.fft.fft(signal)
    num_bins = len(fft)
    mask = np.random.rand(num_bins) > drop_prob
    fft *= mask
    return np.fft.ifft(fft).real

def random_crop(signal, crop_size_ratio=0.9):
    """Randomly crop a portion and resize back to original length."""
    crop_size = int(len(signal) * crop_size_ratio)
    start = np.random.randint(0, len(signal) - crop_size)
    cropped = signal[start:start+crop_size]
    return resample(cropped, len(signal))


In [5]:
class AugmentedECGDataset(Dataset):
    def __init__(self, signals, labels=None, augment=False):
        self.signals = signals
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        signal = self.signals[idx].astype(np.float32)
        if self.augment:
            signal = amplitude_scaling(signal)
            # signal = add_gaussian_noise(signal)
            signal = time_shift(signal)
            signal = time_stretch(signal)
            signal = frequency_dropout(signal)
            signal = random_crop(signal)
        signal = torch.tensor(signal, dtype=torch.float32)
        length = len(signal)
        if self.labels is not None:
            return signal, length, self.labels[idx]
        else:
            return signal, length


In [ ]:
# ✅ Cell 5: Data Loaders
from torch.nn.utils.rnn import pad_sequence

def collate_fn_train(batch):
    signals, lengths, labels = zip(*batch)
    signals = pad_sequence(signals, batch_first=True)
    lengths = torch.tensor(lengths)
    labels = torch.tensor(labels)
    return signals, lengths, labels

def collate_fn_test(batch):
    signals, lengths = zip(*batch)
    signals = pad_sequence(signals, batch_first=True)
    lengths = torch.tensor(lengths)
    return signals, lengths

train_loader = DataLoader(
    AugmentedECGDataset(X_train_split, y_train_split, augment=True),
    batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn_train)

val_loader = DataLoader(
    AugmentedECGDataset(X_val_split, y_val_split, augment=False),
    batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_train)

In [7]:
# ✅ Cell 6: Load Model Class (from 1.2)
class ECG_1D_CNN_Enhanced(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.conv_blocks = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(1, 32, 7, padding=3), nn.BatchNorm1d(32), nn.ReLU(),
                nn.MaxPool1d(2), nn.Dropout(0.2)
            ),
            nn.Sequential(
                nn.Conv1d(32, 64, 11, padding=5), nn.BatchNorm1d(64), nn.ReLU(),
                nn.MaxPool1d(2), nn.Dropout(0.2)
            ),
            nn.Sequential(
                nn.Conv1d(64, 128, 15, padding=7), nn.BatchNorm1d(128), nn.ReLU(),
                nn.MaxPool1d(2), nn.Dropout(0.3)
            ),
            nn.Sequential(
                nn.Conv1d(128, 256, 21, padding=10), nn.BatchNorm1d(256), nn.ReLU(),
                nn.AdaptiveAvgPool1d(1), nn.Dropout(0.4)
            )
        ])
        self.classifier = nn.Sequential(
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.5), nn.Linear(128, num_classes)
        )

    def forward(self, x, lengths=None):
        x = (x - x.mean(dim=1, keepdim=True)) / (x.std(dim=1, keepdim=True) + 1e-8)
        x = x.unsqueeze(1)
        for block in self.conv_blocks:
            x = block(x)
        x = x.squeeze(-1)
        return self.classifier(x)


In [8]:
# ✅ Cell 7: Training Loop
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_split), y=y_train_split)
class_weights = torch.tensor(class_weights, dtype=torch.float32, device=DEVICE)

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    def forward(self, inputs, targets):
        ce = nn.functional.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce)
        return (self.alpha * (1 - pt) ** self.gamma * ce).mean()

def train_model(model, train_loader, val_loader, epochs=EPOCHS):
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    criterion = FocalLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', 0.5, 3)

    best_f1 = 0
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for signals, lengths, labels in train_loader:
            signals, labels = signals.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(signals)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for signals, lengths, labels in val_loader:
                signals, labels = signals.to(DEVICE), labels.to(DEVICE)
                outputs = model(signals)
                preds = torch.argmax(outputs, dim=1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())
        f1 = f1_score(y_true, y_pred, average='macro')
        scheduler.step(f1)
        print(f"Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f}, Val F1={f1:.4f}")
        
    return model

In [ ]:
# ✅ Cell 8: Train & Evaluate
model = ECG_1D_CNN_Enhanced().to(DEVICE)
model = train_model(model, train_loader, val_loader)

Epoch 1: Loss=0.5768, Val F1=0.1853
Epoch 2: Loss=0.4719, Val F1=0.1930
Epoch 3: Loss=0.4511, Val F1=0.2169
Epoch 4: Loss=0.4328, Val F1=0.2751
Epoch 5: Loss=0.4183, Val F1=0.2737
Epoch 6: Loss=0.4072, Val F1=0.2860


In [ ]:
# ✅ Cell 9: Save Predictions on Test Set
test_loader = DataLoader(
    AugmentedECGDataset(X_test, augment=False),
    batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda b: pad_sequence([s for s, _ in b], batch_first=True)
)

model.eval()
test_preds = []
with torch.no_grad():
    for signals in test_loader:
        signals = signals.to(DEVICE)
        outputs = model(signals)
        preds = torch.argmax(outputs, dim=1)
        test_preds.extend(preds.cpu().numpy())

pd.DataFrame({'id': np.arange(len(test_preds)), 'class': test_preds}).to_csv('augment.csv', index=False)
print("✅ Predictions saved to 'augment.csv'")